In [1]:
deck_columns = ['gameMode_name']+[f'team_0_cards_{x}_name' for x in range(8)]+[f'opponent_0_cards_{x}_name' for x in range(8)]+['team_0_crowns']+['opponent_0_crowns']
deck_columns

['gameMode_name',
 'team_0_cards_0_name',
 'team_0_cards_1_name',
 'team_0_cards_2_name',
 'team_0_cards_3_name',
 'team_0_cards_4_name',
 'team_0_cards_5_name',
 'team_0_cards_6_name',
 'team_0_cards_7_name',
 'opponent_0_cards_0_name',
 'opponent_0_cards_1_name',
 'opponent_0_cards_2_name',
 'opponent_0_cards_3_name',
 'opponent_0_cards_4_name',
 'opponent_0_cards_5_name',
 'opponent_0_cards_6_name',
 'opponent_0_cards_7_name',
 'team_0_crowns',
 'opponent_0_crowns']

In [2]:
import os
import pandas as pd

df_match = pd.DataFrame()
for csv_file in os.listdir('CRL_matches'):
    df = pd.read_csv('CRL_matches/'+csv_file)
    #display(df[deck_columns])
    df_match = pd.concat([df_match, df[deck_columns]])
df_match.to_csv('CRL_1_raw.csv')

## EDA

In [8]:
import pandas as pd
import numpy as np

df = pd.read_csv('CRL_1_raw.csv').drop('Unnamed: 0', axis=1)
df.gameMode_name.unique()

array(['Ladder', 'Friendly', 'RAMRIDER_Rage_Buff_Tournament',
       'Draft_Competitive', 'Challenge_AllCards_EventDeck', 'TeamVsTeam',
       'TripleElixir_Ladder', 'MirrorDeck_Friendly', 'Showdown_Ladder',
       'ClanWar_BoatBattle', 'TeamVsTeamLadder', 'Ladder_CrownRush',
       'TeamVsTeam_TripleElixir_Friendly', 'Touchdown_Draft',
       'CW_Battle_1v1', 'Ladder_GoldRush', 'DoubleDeck_Tournament',
       'CW_Duel_1v1', 'Tournament', 'DoubleElixir_AllCards_EventDeck',
       '7xElixir_Friendly', 'Heist_Friendly', 'Overtime_Tournament',
       'TripleElixir_Friendly', 'RampUpElixir_Friendly', 'DraftMode',
       'TripleElixir_Tournament', 'ClassicDecks_Friendly',
       'Friendly_FixedDeckOrder', 'Event_ArcherQueen_Tease',
       'Event_SkeletonKing_Tease', 'Touchdown',
       'Overtime_AllCards_EventDeck', 'TeamVsTeam_Touchdown_Draft',
       'Overtime_Friendly', 'DoubleElixir_Friendly', 'Rage_Friendly',
       'DraftModeInsane'], dtype=object)

In [9]:
#only select proper game modes
df_clean = df[(df['gameMode_name']=='Ladder') | (df['gameMode_name']=='Friendly') | (df['gameMode_name']=='Overtime_Friendly') | (df['gameMode_name']=='Tournament') | (df['gameMode_name']=='Ladder_CrownRush') | (df['gameMode_name']=='Ladder_GoldRush') | (df['gameMode_name']=='CW_Battle_1v1') | (df['gameMode_name']=='CW_Duel_1v1')]

#only select decks, crowns are irrelevant
df_1 = df_clean[[
    'team_0_cards_0_name', 
    'team_0_cards_1_name', 
    'team_0_cards_2_name', 
    'team_0_cards_3_name', 
    'team_0_cards_4_name', 
    'team_0_cards_5_name', 
    'team_0_cards_6_name', 
    'team_0_cards_7_name']]
df_1.columns = list(range(8))

df_2 = df_clean[[
    'opponent_0_cards_0_name', 
    'opponent_0_cards_1_name',
    'opponent_0_cards_2_name', 
    'opponent_0_cards_3_name',
    'opponent_0_cards_4_name', 
    'opponent_0_cards_5_name',
    'opponent_0_cards_6_name', 
    'opponent_0_cards_7_name']]
df_2.columns = list(range(8))

#put everything together
df_clean = pd.concat([df_1, df_2], axis=0)
df_clean
df_clean.to_csv('CRL_2_final.csv')

In [10]:
#index-based encoding
from sklearn import preprocessing
codex = preprocessing.LabelEncoder()
codex = codex.fit(pd.unique(df_clean.values.ravel('K'))) #all unique values in a dataset
#codex.transform(['Battle Healer'])
#codex.inverse_transform([10])

df_index = df_clean.apply(lambda x : codex.transform(x))
df_index

,0,1,2,3,4,5,6,7
0,79,56,61,30,5,62,6,53
1,79,56,61,30,53,62,6,97
2,79,56,61,30,5,62,6,53
3,79,56,61,30,5,62,6,53
4,79,56,61,30,5,62,6,53
...,...,...,...,...,...,...,...,...
16563,25,26,5,50,0,83,6,96
16564,22,14,72,40,91,6,98,60
16565,53,51,30,54,72,95,15,91
16566,74,24,5,11,18,30,104,62


In [11]:
#one_hot encoding
row_list = list()
#len(pd.unique(df_index.values.ravel('K')))
for row in range(df_index.shape[0]):
    a = np.array(df_index.iloc[row])
    b = np.zeros((a.size, 106))
    b[np.arange(a.size),a] = 1
    row_list.append(np.sum(b, axis=0))

df_onehot = pd.DataFrame(row_list) #.astype(int)
df_onehot.columns = [codex.inverse_transform([x])[0] for x in list(df_onehot.columns)]
df_onehot

,Archer Queen,Archers,Arrows,Baby Dragon,Balloon,Bandit,Barbarian Barrel,Barbarian Hut,Barbarians,Bats,...,Three Musketeers,Tombstone,Tornado,Valkyrie,Wall Breakers,Witch,Wizard,X-Bow,Zap,Zappies
0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23671,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23672,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23673,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23674,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [13]:
df_clean

,0,1,2,3,4,5,6,7
0,Ram Rider,Inferno Dragon,Lumberjack,Fireball,Bandit,Magic Archer,Barbarian Barrel,Ice Golem
1,Ram Rider,Inferno Dragon,Lumberjack,Fireball,Ice Golem,Magic Archer,Barbarian Barrel,Tombstone
2,Ram Rider,Inferno Dragon,Lumberjack,Fireball,Bandit,Magic Archer,Barbarian Barrel,Ice Golem
3,Ram Rider,Inferno Dragon,Lumberjack,Fireball,Bandit,Magic Archer,Barbarian Barrel,Ice Golem
4,Ram Rider,Inferno Dragon,Lumberjack,Fireball,Bandit,Magic Archer,Barbarian Barrel,Ice Golem
...,...,...,...,...,...,...,...,...
16563,Elite Barbarians,Elixir Collector,Bandit,Heal Spirit,Archer Queen,Royal Ghost,Barbarian Barrel,Three Musketeers
16564,Electro Giant,Bowler,Musketeer,Goblin Cage,Skeletons,Barbarian Barrel,Tornado,Lightning
16565,Ice Golem,Hog Rider,Fireball,Ice Spirit,Musketeer,The Log,Cannon,Skeletons
16566,P.E.K.K.A,Electro Wizard,Bandit,Battle Ram,Dark Prince,Fireball,Zap,Magic Archer


In [14]:
pd.DataFrame([[list(df_clean.iloc[x].values)] for x in range(len(df_clean))])

,0
0,"[Ram Rider, Inferno Dragon, Lumberjack, Fireba..."
1,"[Ram Rider, Inferno Dragon, Lumberjack, Fireba..."
2,"[Ram Rider, Inferno Dragon, Lumberjack, Fireba..."
3,"[Ram Rider, Inferno Dragon, Lumberjack, Fireba..."
4,"[Ram Rider, Inferno Dragon, Lumberjack, Fireba..."
...,...
23671,"[Elite Barbarians, Elixir Collector, Bandit, H..."
23672,"[Electro Giant, Bowler, Musketeer, Goblin Cage..."
23673,"[Ice Golem, Hog Rider, Fireball, Ice Spirit, M..."
23674,"[P.E.K.K.A, Electro Wizard, Bandit, Battle Ram..."


In [15]:
#implode horizontally
df_onehot_compressed = pd.DataFrame([[df_onehot.iloc[x].values] for x in range(len(df_onehot))])
df_onehot_compressed.index = df_onehot_compressed.index.set_names(['_id'])
df_onehot_compressed.reset_index()
df_onehot_compressed.columns = ['deck']
df_onehot_compressed['original'] = pd.DataFrame([[list(df_clean.iloc[x].values)] for x in range(len(df_clean))]).values
df_onehot_compressed

,deck,original
_id,,
0,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, ...","[Ram Rider, Inferno Dragon, Lumberjack, Fireba..."
1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","[Ram Rider, Inferno Dragon, Lumberjack, Fireba..."
2,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, ...","[Ram Rider, Inferno Dragon, Lumberjack, Fireba..."
3,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, ...","[Ram Rider, Inferno Dragon, Lumberjack, Fireba..."
4,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, ...","[Ram Rider, Inferno Dragon, Lumberjack, Fireba..."
...,...,...
23671,"[1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, ...","[Elite Barbarians, Elixir Collector, Bandit, H..."
23672,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","[Electro Giant, Bowler, Musketeer, Goblin Cage..."
23673,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[Ice Golem, Hog Rider, Fireball, Ice Spirit, M..."


In [ ]:
dict_cards

In [30]:
df_onehot_compressed['links'] = df_onehot_compressed['original'].apply(lambda x: [dictionary[a] for a in x])

In [ ]:
#prepare for upload
df_ready = df_onehot_compressed.copy()
df_ready.index = df_ready.index.set_names(['_id'])
df_ready = df_ready.reset_index()
df_ready['deck'] = df_ready['deck'].apply(lambda x: list(x))
df_ready.columns = ['_id', 'deck_vector_', 'deck_original', 'links']
df_ready = df_ready.to_dict(orient='records')
df_ready

## image association
https://statsroyale.com/cards

In [38]:
from Levenshtein import distance
import os

list_cards = list(pd.unique(df_clean.values.ravel('K')))
list_img = os.listdir('CRL_images')

In [20]:
dict_cards = {
    'Archer Queen':'archerqueen',
    'Archers':'archers',
    'Arrows':'order_volley',
    'Ram Rider':'ram_rider',
    'Baby Dragon':'baby_dragon',
    'Balloon':'chr_balloon',
    'Bandit':'bandit',
    'Barbarian Barrel':'barb_barrel',
    'Barbarian Hut':'barbarian_hut',
    'Barbarians':'barbarians',
    'Bats':'bats',
    'Battle Healer':'battle_healer',
    'Battle Ram':'battle_ram',
    'Bomb Tower':'bomb_tower',
    'Bomber':'bomber',
    'Bowler':'bowler',
    'Cannon':'chaos_cannon',
    'Cannon Cart':'cannon_cart',
    'Clone':'copy',
    'Dark Prince':'dark_prince',
    'Dart Goblin':'blowdart_goblin',
    'Earthquake':'earthquake',
    'Electro Dragon':'electro_dragon',
    'Electro Giant':'electrogiant',
    'Electro Spirit':'electrospirit',
    'Electro Wizard':'electro_wizard',
    'Elite Barbarians':'angry_barbarian',
    'Elixir Collector':'building_elixir_collector',
    'Elixir Golem':'elixir_golem',
    'Executioner':'executioner',
    'Fire Spirit':'fire_spirits',
    'Fireball':'fire_fireball',
    'Firecracker':'firecracker',
    'Fisherman':'fisherman',
    'Flying Machine':'flying_machine',
    'Freeze':'freeze',
    'Furnace':'firespirit_hut',
    'Giant':'giant',
    'Giant Skeleton':'giant_skeleton',
    'Giant Snowball':'snowball',
    'Goblin Barrel':'goblin_barrel',
    'Goblin Cage':'goblin_cage',
    'Goblin Drill':'goblindrill',
    'Goblin Gang':'goblin_gang',
    'Goblin Giant':'goblin_giant',
    'Goblin Hut':'fire_furnace',
    'Goblins':'goblins',
    'Golden Knight':'goldenknight',
    'Golem':'chr_golem',
    'Graveyard':'graveyard',
    'Guards':'skeleton_warriors',
    'Heal Spirit':'healspirit',
    'Hog Rider':'hog_rider',
    'Hunter':'hunter',
    'Ice Golem':'ice_golem',
    'Ice Spirit':'snow_spirits',
    'Ice Wizard':'ice_wizard',
    'Inferno Dragon':'inferno_dragon',
    'Inferno Tower':'building_inferno',
    'Knight':'knight',
    'Lava Hound':'lava_hound',
    'Lightning':'lightning',
    'Lumberjack':'rage_barbarian',
    'Magic Archer':'magic_archer',
    'Mega Knight':'mega_knight',
    'Mega Minion':'mega_minion',
    'Miner':'miner',
    'Mini P.E.K.K.A':'mini_pekka',
    'Minion Horde':'minion_horde',
    'Minions':'minion',
    'Mirror':'mirror',
    'Mortar':'building_mortar',
    'Mother Witch':'motherwitch',
    'Musketeer':'musketeer',
    'Night Witch':'dark_witch',
    'P.E.K.K.A':'pekka',
    'Poison':'poison',
    'Prince':'prince',
    'Princess':'princess',
    'Rage':'rage',
    'Ram Rider':'ram_rider',
    'Rascals':'rascals',
    'Rocket':'rocket',
    'Royal Delivery':'royal_delivery',
    'Royal Ghost':'ghost',
    'Royal Giant':'royal_giant',
    'Royal Hogs':'royal_hog',
    'Royal Recruits':'royal_recruits',
    'Skeleton Army':'skeleton_horde',
    'Skeleton Barrel':'skeleton_balloon',
    'Skeleton Dragons':'skeletondragon',
    'Skeleton King':'skeletonking',
    'Skeletons':'skeletons',
    'Sparky':'zapMachine',
    'Spear Goblins':'goblin_archer',
    'Tesla':'building_tesla',
    'The Log':'the_log',
    'Three Musketeers':'three_musketeers',
    'Tombstone':'tombstone',
    'Tornado':'tornado',
    'Valkyrie':'valkyrie',
    'Wall Breakers':'wallbreaker',
    'Witch':'chr_witch',
    'Wizard':'wizard',
    'X-Bow':'building_xbow',
    'Zap':'zap',
    'Zappies':'zappies'
}

In [28]:
list_1 = list(dict_cards.keys())
list_2 = ['https://raw.githubusercontent.com/arditoibryan/Projects/master/20210113_CRL_challenge/CRL_images/'+x+'.png' for x in list(dict_cards.values())]
dictionary = dict(zip(list_1, list_2))
dictionary

{'Archer Queen': 'https://raw.githubusercontent.com/arditoibryan/Projects/master/20210113_CRL_challenge/CRL_images/archerqueen.png',
 'Archers': 'https://raw.githubusercontent.com/arditoibryan/Projects/master/20210113_CRL_challenge/CRL_images/archers.png',
 'Arrows': 'https://raw.githubusercontent.com/arditoibryan/Projects/master/20210113_CRL_challenge/CRL_images/order_volley.png',
 'Ram Rider': 'https://raw.githubusercontent.com/arditoibryan/Projects/master/20210113_CRL_challenge/CRL_images/ram_rider.png',
 'Baby Dragon': 'https://raw.githubusercontent.com/arditoibryan/Projects/master/20210113_CRL_challenge/CRL_images/baby_dragon.png',
 'Balloon': 'https://raw.githubusercontent.com/arditoibryan/Projects/master/20210113_CRL_challenge/CRL_images/chr_balloon.png',
 'Bandit': 'https://raw.githubusercontent.com/arditoibryan/Projects/master/20210113_CRL_challenge/CRL_images/bandit.png',
 'Barbarian Barrel': 'https://raw.githubusercontent.com/arditoibryan/Projects/master/20210113_CRL_challen

## clustering

In [17]:
#   SDK
import relevanceai

project = "michelangioloma" #'638c33786033a2ebd49f'
api_key = "TDFvdm9Yd0J1M3VVTEcyQTV3VWs6b3RnRXJnUW9SQ2kwMVVJWVZ2VzVtZw" #'TGx6UGNIWUJ5R0lBMktUNTRsbjM6VElZRzhaRjlRUnVYY3RfQ0pISXd1QQ'
client = relevanceai.Client(project, api_key)
client.base_url = "https://gateway-api-aueast.relevance.ai/latest"

In [295]:
#client.datasets.delete(dataset_id='crl_matches')

{'status': 'complete', 'message': 'crl_matches deleted'}

In [36]:
#upload df
client.datasets.bulk_insert(dataset_id='crl_matches', documents=df_ready)

{'inserted': 23676, 'failed_documents': []}

In [37]:
client.datasets.schema('crl_matches')

{'deck_original': 'text',
 'deck_vector_': {'vector': 106},
 'insert_date_': 'date',
 'links': 'text'}

In [ ]:
# Vector field based on which clustering is done - (Currently only one vector is supported)
vector_field = 'descriptiontextmulti_vector_'

#local or remote?
centroids = client.vector_tools.cluster.kmeans_cluster(
    dataset_id = 'crl_matches', 
    vector_fields=['deck_vector_'],
    k = 12)

#creates clusters but only gives the centroids
#clustering results is uploaded on the database

In [38]:
client.datasets.schema('crl_matches')

{'deck_original': 'text',
 'deck_vector_': {'vector': 106},
 'insert_date_': 'date',
 'links': 'text'}

In [6]:
client.services.cluster.centroids.list_closest_to_center(
  dataset_id='crl_matches',
  vector_fields=["deck_vector_"],
  page_size=20,
  #cluster_ids=[], # Leave this as an empty list if you want all of the clusters
  alias="kmeans_12" #change to 'kmeans_10' 
)

You can now visit the dashboard at https://cloud.relevance.ai/sdk/cluster/centroids/closest


{'Cluster-0': [{'_id': '12269',
   'deck_original': ['Barbarians',
    'Mega Minion',
    'Zap',
    'Fireball',
    'Lava Hound',
    'Balloon',
    'Tombstone',
    'Skeleton Dragons'],
   'insert_date_': '2022-01-06T00:13:47.414Z',
   '_cluster_': {},
   '_search_score': 0.8797486999999999},
  {'_id': '12201',
   'deck_original': ['Lava Hound',
    'Balloon',
    'Tombstone',
    'Skeleton Dragons',
    'Mega Minion',
    'Fireball',
    'Barbarians',
    'Zap'],
   'insert_date_': '2022-01-06T00:13:47.434Z',
   '_cluster_': {},
   '_search_score': 0.8797486999999999},
  {'_id': '12151',
   'deck_original': ['Lava Hound',
    'Balloon',
    'Tombstone',
    'Skeleton Dragons',
    'Mega Minion',
    'Fireball',
    'Barbarians',
    'Zap'],
   'insert_date_': '2022-01-06T00:13:47.444Z',
   '_cluster_': {},
   '_search_score': 0.8797486999999999},
  {'_id': '11908',
   'deck_original': ['Lava Hound',
    'Balloon',
    'Tombstone',
    'Skeleton Dragons',
    'Mega Minion',
    'Fire